# PaBiRoboy dynamic equations

First, import the necessary functions from SymPy that will allow us to construct time varying vectors in the reference frames.

In [1]:
from __future__ import print_function, division
from sympy import symbols, simplify, Matrix
from sympy import trigsimp
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point, inertia, RigidBody, KanesMethod
from numpy import deg2rad, rad2deg, array, zeros, linspace
from sympy.physics.vector import init_vprinting, vlatex
import numpy as np
from scipy.integrate import odeint
from pydy.codegen.ode_function_generators import generate_ode_function
%matplotlib inline
from matplotlib.pyplot import plot, legend, xlabel, ylabel, rcParams
rcParams['figure.figsize'] = (14.0, 6.0)
import rospy

SymPy has a rich printing system. Here we initialize printing so that all of the mathematical equations are rendered in standard mathematical notation.

In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

## Interial frames

In [3]:
inertial_frame = ReferenceFrame('I')
lower_leg_left_frame = ReferenceFrame('R_1')
upper_leg_left_frame = ReferenceFrame('R_2')
hip_frame = ReferenceFrame('R_3')
upper_leg_right_frame = ReferenceFrame('R_4')
lower_leg_right_frame = ReferenceFrame('R_5')

## Angles

In [4]:
theta0, theta1, theta2, theta3, theta4, theta5 = dynamicsymbols('theta0, theta1, theta2, theta3, theta4, theta5')

In [5]:
lower_leg_left_frame.orient(inertial_frame, 'Axis', (theta0, inertial_frame.z))
lower_leg_left_frame.dcm(inertial_frame)

Matrix([
[ cos(theta0), sin(theta0), 0],
[-sin(theta0), cos(theta0), 0],
[           0,           0, 1]])

In [6]:
upper_leg_left_frame.orient(lower_leg_left_frame, 'Axis', (theta1, lower_leg_left_frame.z))
simplify(upper_leg_left_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta1), sin(theta0 + theta1), 0],
[-sin(theta0 + theta1), cos(theta0 + theta1), 0],
[                    0,                    0, 1]])

In [7]:
hip_frame.orient(upper_leg_left_frame, 'Axis', (theta2, upper_leg_left_frame.z))
simplify(hip_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta1 + theta2), sin(theta0 + theta1 + theta2), 0],
[-sin(theta0 + theta1 + theta2), cos(theta0 + theta1 + theta2), 0],
[                             0,                             0, 1]])

In [8]:
upper_leg_right_frame.orient(hip_frame, 'Axis', (theta3, hip_frame.z))
simplify(upper_leg_right_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta1 + theta2 + theta3), sin(theta0 + theta1 + theta2 + theta3), 0],
[-sin(theta0 + theta1 + theta2 + theta3), cos(theta0 + theta1 + theta2 + theta3), 0],
[                                      0,                                      0, 1]])

In [9]:
lower_leg_right_frame.orient(upper_leg_right_frame, 'Axis', (theta4, upper_leg_right_frame.z))
simplify(lower_leg_right_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta1 + theta2 + theta3 + theta4), sin(theta0 + theta1 + theta2 + theta3 + theta4), 0],
[-sin(theta0 + theta1 + theta2 + theta3 + theta4), cos(theta0 + theta1 + theta2 + theta3 + theta4), 0],
[                                               0,                                               0, 1]])

## Points and Locations

In [10]:
ankle_left = Point('AnkleLeft')
knee_left = Point('KneeLeft')
hip_left = Point('HipLeft')
hip_right = Point('HipRight')
knee_right = Point('KneeRight')
ankle_right = Point('AnkleRight')

In [11]:
lower_leg_length, upper_leg_length, hip_length = symbols('l1, l2, l3')

In [12]:
knee_left.set_pos(ankle_left, lower_leg_length * lower_leg_left_frame.y)
knee_left.pos_from(ankle_left).express(inertial_frame).simplify()

- l1*sin(theta0)*I.x + l1*cos(theta0)*I.y

In [13]:
hip_left.set_pos(knee_left, upper_leg_length * upper_leg_left_frame.y)
hip_left.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1))*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1))*I.y

In [14]:
hip_right.set_pos(hip_left, hip_length * hip_frame.y)
hip_right.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1) - l3*sin(theta0 + theta1 + theta2))*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1) + l3*cos(theta0 + theta1 + theta2))*I.y

In [15]:
knee_right.set_pos(hip_right, upper_leg_length * upper_leg_right_frame.y)
knee_right.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3) - l3*sin(theta0 + theta1 + theta2))*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1) + l2*cos(theta0 + theta1 + theta2 + theta3) + l3*cos(theta0 + theta1 + theta2))*I.y

In [16]:
ankle_right.set_pos(knee_right, lower_leg_length * lower_leg_right_frame.y)
ankle_right.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0 + theta1 + theta2 + theta3 + theta4) - l1*sin(theta0) - l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3) - l3*sin(theta0 + theta1 + theta2))*I.x + (l1*cos(theta0 + theta1 + theta2 + theta3 + theta4) + l1*cos(theta0) + l2*cos(theta0 + theta1) + l2*cos(theta0 + theta1 + theta2 + theta3) + l3*cos(theta0 + theta1 + theta2))*I.y

In [17]:
lower_leg_left_com_length = lower_leg_length/2
upper_leg_left_com_length = upper_leg_length/2
hip_com_length = hip_length/2
upper_leg_right_com_length = upper_leg_length/2
lower_leg_right_com_length = lower_leg_length/2

lower_leg_left_mass_center = Point('L_COMleft')
upper_leg_left_mass_center = Point('U_COMleft')
hip_mass_center = Point('H_COMleft')
upper_leg_right_mass_center = Point('U_COMright')
lower_leg_right_mass_center = Point('L_COMright')

In [18]:
lower_leg_left_mass_center.set_pos(ankle_left, lower_leg_left_com_length * lower_leg_left_frame.y)
lower_leg_left_mass_center.pos_from(ankle_left).express(inertial_frame).simplify()

- l1*sin(theta0)/2*I.x + l1*cos(theta0)/2*I.y

In [19]:
upper_leg_left_mass_center.set_pos(knee_left, upper_leg_left_com_length * upper_leg_left_frame.y)
upper_leg_left_mass_center.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1)/2)*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1)/2)*I.y

In [20]:
hip_mass_center.set_pos(hip_left, hip_com_length * hip_frame.y)
hip_mass_center.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1) - l3*sin(theta0 + theta1 + theta2)/2)*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1) + l3*cos(theta0 + theta1 + theta2)/2)*I.y

In [21]:
upper_leg_right_mass_center.set_pos(hip_right, upper_leg_right_com_length * upper_leg_right_frame.y)
upper_leg_right_mass_center.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3)/2 - l3*sin(theta0 + theta1 + theta2))*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1) + l2*cos(theta0 + theta1 + theta2 + theta3)/2 + l3*cos(theta0 + theta1 + theta2))*I.y

In [22]:
lower_leg_right_mass_center.set_pos(knee_right, lower_leg_right_com_length * lower_leg_right_frame.y)
lower_leg_right_mass_center.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0 + theta1 + theta2 + theta3 + theta4)/2 - l1*sin(theta0) - l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3) - l3*sin(theta0 + theta1 + theta2))*I.x + (l1*cos(theta0 + theta1 + theta2 + theta3 + theta4)/2 + l1*cos(theta0) + l2*cos(theta0 + theta1) + l2*cos(theta0 + theta1 + theta2 + theta3) + l3*cos(theta0 + theta1 + theta2))*I.y

# Jacobian

#### We want to control the hip com position. The hip com seen from ankle_left in inertial_frame is:

In [23]:
F0 = ankle_left.pos_from(hip_mass_center).express(inertial_frame).simplify().to_matrix(inertial_frame)
F0 = Matrix([F0[0], F0[1]])
F0

Matrix([
[ l1*sin(theta0) + l2*sin(theta0 + theta1) + l3*sin(theta0 + theta1 + theta2)/2],
[-l1*cos(theta0) - l2*cos(theta0 + theta1) - l3*cos(theta0 + theta1 + theta2)/2]])

In [24]:
J_hipCOM = F0.jacobian([theta0, theta1, theta2, theta3, theta4])
J_hipCOM

Matrix([
[l1*cos(theta0) + l2*cos(theta0 + theta1) + l3*cos(theta0 + theta1 + theta2)/2, l2*cos(theta0 + theta1) + l3*cos(theta0 + theta1 + theta2)/2, l3*cos(theta0 + theta1 + theta2)/2, 0, 0],
[l1*sin(theta0) + l2*sin(theta0 + theta1) + l3*sin(theta0 + theta1 + theta2)/2, l2*sin(theta0 + theta1) + l3*sin(theta0 + theta1 + theta2)/2, l3*sin(theta0 + theta1 + theta2)/2, 0, 0]])

#### We want to constraint the ankle_right stay at the initial step size

In [25]:
step_length = symbols('L')

In [26]:
F1 = ankle_right.pos_from(ankle_left).express(inertial_frame).simplify().to_matrix(inertial_frame)
F1 = Matrix([F1[0], F1[1]])
F1

Matrix([
[-l1*sin(theta0 + theta1 + theta2 + theta3 + theta4) - l1*sin(theta0) - l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3) - l3*sin(theta0 + theta1 + theta2)],
[ l1*cos(theta0 + theta1 + theta2 + theta3 + theta4) + l1*cos(theta0) + l2*cos(theta0 + theta1) + l2*cos(theta0 + theta1 + theta2 + theta3) + l3*cos(theta0 + theta1 + theta2)]])

In [27]:
offset = Matrix([step_length, F0[1]])
offset

Matrix([
[                                                                             L],
[-l1*cos(theta0) - l2*cos(theta0 + theta1) - l3*cos(theta0 + theta1 + theta2)/2]])

In [28]:
constraint = F1 - offset
J_ankleRight = constraint.jacobian([theta0, theta1, theta2, theta3, theta4])
J_ankleRight

Matrix([
[        -l1*cos(theta0 + theta1 + theta2 + theta3 + theta4) - l1*cos(theta0) - l2*cos(theta0 + theta1) - l2*cos(theta0 + theta1 + theta2 + theta3) - l3*cos(theta0 + theta1 + theta2),       -l1*cos(theta0 + theta1 + theta2 + theta3 + theta4) - l2*cos(theta0 + theta1) - l2*cos(theta0 + theta1 + theta2 + theta3) - l3*cos(theta0 + theta1 + theta2),     -l1*cos(theta0 + theta1 + theta2 + theta3 + theta4) - l2*cos(theta0 + theta1 + theta2 + theta3) - l3*cos(theta0 + theta1 + theta2), -l1*cos(theta0 + theta1 + theta2 + theta3 + theta4) - l2*cos(theta0 + theta1 + theta2 + theta3), -l1*cos(theta0 + theta1 + theta2 + theta3 + theta4)],
[-l1*sin(theta0 + theta1 + theta2 + theta3 + theta4) - 2*l1*sin(theta0) - 2*l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3) - 3*l3*sin(theta0 + theta1 + theta2)/2, -l1*sin(theta0 + theta1 + theta2 + theta3 + theta4) - 2*l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3) - 3*l3*sin(theta0 + theta1 + theta2)/2, -l1*sin(

In [29]:
J = J_hipCOM.col_join(J_ankleRight)
J

Matrix([
[                                                                                                        l1*cos(theta0) + l2*cos(theta0 + theta1) + l3*cos(theta0 + theta1 + theta2)/2,                                                                                                       l2*cos(theta0 + theta1) + l3*cos(theta0 + theta1 + theta2)/2,                                                                                                     l3*cos(theta0 + theta1 + theta2)/2,                                                                                               0,                                                   0],
[                                                                                                        l1*sin(theta0) + l2*sin(theta0 + theta1) + l3*sin(theta0 + theta1 + theta2)/2,                                                                                                       l2*sin(theta0 + theta1) + l3*sin(theta0 + theta1 + theta2)/2,         

In [30]:
values = {lower_leg_length: 0.4, upper_leg_length: 0.54, hip_length: 0.2, theta0: 0.1, theta1: 0.1, theta2: 0.1, theta3: 0.1, theta4: 0.1}
J.subs(values).pinv()

Matrix([
[ 3.24197721742672, -16.2764183239215,   -1.295692049053,  2.54805972845702],
[-2.53918372183114,   22.548228436548,  3.20564610197954,  -6.3041042368892],
[-7.63496793145106,  26.7927095751062, -7.09272224914093,  13.9482834222648],
[ 0.59375470240343, -12.6921656291146,  -1.8003428317574,  2.33036301670791],
[ 14.3570793965002, -58.2361185617551,  15.4391222509288, -33.0394894146124]])

In [31]:
kp=5
hip_desired = Matrix([0.5, 0.5])
ankle_right_desired = Matrix([1.5, 0])
x_des = Matrix([hip_desired, ankle_right_desired])
x_des

Matrix([
[0.5],
[0.5],
[1.5],
[  0]])

In [ ]:
ankle_left_pos_equation = ankle_left.pos_from(ankle_left).express(inertial_frame).simplify()
knee_left_pos_equation = knee_left.pos_from(ankle_left).express(inertial_frame).simplify()
hip_left_pos_equation = hip_left.pos_from(ankle_left).express(inertial_frame).simplify()
hip_right_pos_equation = hip_right.pos_from(ankle_left).express(inertial_frame).simplify()
knee_right_pos_equation = knee_right.pos_from(ankle_left).express(inertial_frame).simplify()
ankle_right_pos_equation = ankle_right.pos_from(ankle_left).express(inertial_frame).simplify()
hip_mass_center_pos_equation = ankle_left.pos_from(hip_mass_center).express(inertial_frame).simplify()

In [ ]:
from roboy_communication_middleware.msg import JointStatus
from roboy_communication_middleware.msg import JointCommand
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point as geometry_msgsPoint
rospy.init_node('PaBiRoboy_dynamics', anonymous=True)

class PaBiRoboy_DanceController:
    jointCommand_pub = rospy.Publisher('/roboy/middleware/JointCommand', JointCommand, queue_size=1)
    visualization_pub = rospy.Publisher('/visualization_marker', Marker, queue_size=1)
    msg_viz = Marker()
    def __init__(self):
        self.msg_viz.header.frame_id = "world"
        self.msg_viz.type = Marker.ARROW
        self.msg_viz.scale.x = 0.005
        self.msg_viz.scale.y = 0.05
        self.msg_viz.scale.z = 0.05
        self.msg_viz.color.a = 0.5
        self.msg_viz.color.r = 1.0
        self.msg_viz.color.g = 0.0
        self.msg_viz.color.b = 0.0
        self.listener()
    def ros_callback(self, data):
        q1 = deg2rad(data.relAngles[1]/4096.0*360.0)
        q2 = deg2rad(data.relAngles[0]/4096.0*360.0)
        q3 = deg2rad(data.relAngles[2]/4096.0*360.0)
        q4 = deg2rad(data.relAngles[3]/4096.0*360.0)
        print (q1, q2, q3, q4)
        values = {lower_leg_length: 0.4, upper_leg_length: 0.54, hip_length: 0.2, theta0: deg2rad(30), theta1: q1 , theta2: q2, theta3: q3, theta4: q4}
        Jpinv = J.subs(values).pinv()
        x_mes_hip = hip_mass_center_pos_equation.subs(values).to_matrix(inertial_frame)
        x_mes_ankle_right = ankle_right_pos_equation.subs(values).to_matrix(inertial_frame)
        x_mes = Matrix([x_mes_hip[0], x_mes_hip[1], x_mes_ankle_right[0], x_mes_ankle_right[1]])
        dq=Jpinv*( x_des + kp*(x_des - x_mes ))
        msg = JointCommand()
        msg.dq = dq
        self.jointCommand_pub.publish(msg)
        
        self.msg_viz.header.stamp = rospy.Time.now()
        ankle_left_pos = ankle_left_pos_equation.subs(values).to_matrix(inertial_frame)
        knee_left_pos = knee_left_pos_equation.subs(values).to_matrix(inertial_frame)
        hip_left_pos = hip_left_pos_equation.subs(values).to_matrix(inertial_frame)
        hip_right_pos = hip_right_pos_equation.subs(values).to_matrix(inertial_frame)
        knee_right_pos = knee_right_pos_equation.subs(values).to_matrix(inertial_frame)
        ankle_right_pos = ankle_right_pos_equation.subs(values).to_matrix(inertial_frame)

        self.msg_viz.id = 0
        del self.msg_viz.points[:]
        self.msg_viz.points.append(geometry_msgsPoint(ankle_left_pos[0],ankle_left_pos[1],ankle_left_pos[2]))
        self.msg_viz.points.append(geometry_msgsPoint(knee_left_pos[0],knee_left_pos[1],knee_left_pos[2]))
        self.visualization_pub.publish(self.msg_viz)

        self.msg_viz.id = 1
        del self.msg_viz.points[:]
        self.msg_viz.points.append(geometry_msgsPoint(knee_left_pos[0],knee_left_pos[1],knee_left_pos[2]))
        self.msg_viz.points.append(geometry_msgsPoint(hip_left_pos[0],hip_left_pos[1],hip_left_pos[2]))
        self.visualization_pub.publish(self.msg_viz)

        self.msg_viz.id = 2
        del self.msg_viz.points[:]
        self.msg_viz.points.append(geometry_msgsPoint(hip_left_pos[0],hip_left_pos[1],hip_left_pos[2]))
        self.msg_viz.points.append(geometry_msgsPoint(hip_right_pos[0],hip_right_pos[1],hip_right_pos[2]))
        self.visualization_pub.publish(self.msg_viz)

        self.msg_viz.id = 3
        del self.msg_viz.points[:]
        self.msg_viz.points.append(geometry_msgsPoint(hip_right_pos[0],hip_right_pos[1],hip_right_pos[2]))
        self.msg_viz.points.append(geometry_msgsPoint(knee_right_pos[0],knee_right_pos[1],knee_right_pos[2]))
        self.visualization_pub.publish(self.msg_viz)

        self.msg_viz.id = 4
        del self.msg_viz.points[:]
        self.msg_viz.points.append(geometry_msgsPoint(knee_right_pos[0],knee_right_pos[1],knee_right_pos[2]))
        self.msg_viz.points.append(geometry_msgsPoint(ankle_right_pos[0],ankle_right_pos[1],ankle_right_pos[2]))
        self.visualization_pub.publish(self.msg_viz)
        
    def listener(self):
        rospy.Subscriber("roboy/middleware/JointStatus", JointStatus, self.ros_callback)
        rospy.spin()


# In[34]:
danceController = PaBiRoboy_DanceController()

4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35957339917 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.0632

4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.0632

4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.3565054376 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71821395613 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167

4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35957339917 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06320415971
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.06167017892
4.35803941838 1.32535940073 2.71667997535 2.0616

4.44701030408 1.44961184455 2.60623335862 2.18285466116
4.44701030408 1.44961184455 2.60469937783 2.18285466116
4.44701030408 1.45114582534 2.60469937783 2.18285466116
4.44854428487 1.45114582534 2.60316539704 2.18438864195
4.44854428487 1.45267980613 2.60316539704 2.18592262274
4.44854428487 1.45267980613 2.60316539704 2.18745660352
4.44854428487 1.45421378692 2.60316539704 2.18745660352
4.44854428487 1.45421378692 2.60163141625 2.18745660352
4.44854428487 1.45421378692 2.60163141625 2.18745660352
4.44854428487 1.4557477677 2.60009743547 2.18899058431
4.44854428487 1.45728174849 2.60009743547 2.18899058431
4.44854428487 1.45728174849 2.59856345468 2.1905245651
4.44854428487 1.45881572928 2.59702947389 2.19205854589
4.45007826566 1.46034971007 2.59702947389 2.19205854589
4.44854428487 1.46188369086 2.5954954931 2.19359252668
4.45007826566 1.46188369086 2.59396151231 2.19512650746
4.45007826566 1.46341767164 2.59242753153 2.19512650746
4.45007826566 1.46495165243 2.59242753153 2.1981944

4.27520445584 1.24559239976 2.79644697632 1.99877696661
4.27367047505 1.24252443819 2.79951493789 1.99724298583
4.27213649426 1.2409904574 2.80104891868 1.99570900504
4.26906853269 1.23945647661 2.80411688025 1.99417502425
4.2675345519 1.23638851504 2.80718484183 1.99417502425
4.26600057111 1.23485453425 2.80871882262 1.99110706268
4.26446659032 1.23332055346 2.81025280341 1.99110706268
4.26139862875 1.23178657267 2.81178678419 1.98957308189
4.25986464796 1.23025259188 2.81332076498 1.9880391011
4.25833066717 1.23025259188 2.81332076498 1.98650512031
4.25833066717 1.23025259188 2.81485474577 1.98650512031
4.25526270559 1.23025259188 2.81485474577 1.98497113952
4.25372872481 1.23025259188 2.81485474577 1.98343715874
4.25066076323 1.23025259188 2.81485474577 1.98190317795
4.25066076323 1.23025259188 2.81485474577 1.98190317795
4.24912678244 1.23025259188 2.81485474577 1.98036919716
4.24759280166 1.23025259188 2.81485474577 1.98036919716
4.24759280166 1.23025259188 2.81485474577 1.9788352

4.45314622723 1.49869922976 2.56941781971 2.22734010401
4.45314622723 1.49563126819 2.5709518005 2.22580612322
4.45161224644 1.49256330661 2.57401976207 2.22120418086
4.45161224644 1.48949534504 2.57555374286 2.21813621928
4.45007826566 1.48489340267 2.57862170444 2.21506825771
4.45007826566 1.48029146031 2.58168966601 2.21046631534
4.44854428487 1.47568951795 2.58475762759 2.20739835377
4.44854428487 1.47108757558 2.58935956995 2.2027964114
4.44701030408 1.46648563322 2.59242753153 2.19819446904
4.44547632329 1.46188369086 2.5954954931 2.19512650746
4.44547632329 1.4557477677 2.60163141625 2.1905245651
4.4439423425 1.44961184455 2.60623335862 2.18592262274
4.44240836172 1.4434759214 2.60930132019 2.17978669959
4.44087438093 1.43733999825 2.61543724335 2.17518475722
4.43627243857 1.4312040751 2.62003918571 2.17058281486
4.43320447699 1.42506815195 2.62617510886 2.16444689171
4.43013651541 1.41893222879 2.63077705122 2.15831096856
4.42706855384 1.41433028643 2.63537899359 2.15524300698


4.40866078438 1.37904872831 2.66605860935 2.13069931437
4.40252486123 1.37291280516 2.6721945325 2.12302941043
4.39638893808 1.36831086279 2.67679647486 2.11689348728
4.39025301493 1.36217493964 2.68139841722 2.10922358334
4.38258311099 1.35757299728 2.68600035959 2.1015536794
4.37491320705 1.35297105492 2.69060230195 2.09388377546
4.3687772839 1.34683513176 2.69520424432 2.08621387152
4.36264136075 1.34069920861 2.69980618668 2.07854396759
4.35497145681 1.33456328546 2.70747609062 2.06934008286
4.34730155287 1.32842736231 2.71361201377 2.06167017892
4.33963164893 1.32229143916 2.71974793692 2.05553425577
4.33349572578 1.31615551601 2.72588386007 2.04786435183
4.32582582184 1.30848561207 2.73355376401 2.04172842868
4.31968989869 1.30081570813 2.74122366795 2.03405852474
4.31048601396 1.29314580419 2.75042755268 2.0263886208
4.30435009081 1.28547590025 2.75809745662 2.01871871686
4.29514620608 1.27627201552 2.76730134135 2.01411677449
4.28747630214 1.26706813079 2.77803920686 2.00951483

4.31968989869 1.29007784261 2.74889357189 2.0386604671
4.31355397553 1.28394191946 2.75656347583 2.03252454395
4.30435009081 1.27627201552 2.76269939898 2.0263886208
4.29821416766 1.27013609237 2.77036930292 2.02025269765
4.28747630214 1.26246618843 2.77803920686 2.01258279371
4.2798063982 1.25633026528 2.7857091108 2.00491288977
4.27213649426 1.25019434213 2.79337901474 2.0003109474
4.26293260953 1.24405841898 2.80258289947 1.99417502425
4.25526270559 1.23638851504 2.80871882262 1.98957308189
4.24605882087 1.23025259188 2.81945668813 1.98343715874
4.23992289772 1.22258268794 2.82712659207 1.97883521637
4.23225299378 1.21644676479 2.83326251522 1.97423327401
4.22458308984 1.21031084164 2.84093241916 1.96963133165
4.21537920511 1.20417491849 2.84706834232 1.96349540849
4.20924328196 1.20110695691 2.85167028468 1.96042744692
4.20157337802 1.19957297613 2.85473824626 1.95582550455
4.19390347408 1.19803899534 2.85627222704 1.95122356219
4.18930153172 1.19803899534 2.85780620783 1.946621619

## Kinematical Differential Equations

In [ ]:
omega0, omega1, omega2, omega3, omega4 = dynamicsymbols('omega0, omega1, omega2, omega3, omega4')
kinematical_differential_equations = [omega0 - theta0.diff(),
                                      omega1 - theta1.diff(),
                                      omega2 - theta2.diff(),
                                      omega3 - theta3.diff(),
                                      omega4 - theta4.diff(),]

kinematical_differential_equations

## Angular Velocities

In [ ]:
lower_leg_left_frame.set_ang_vel(inertial_frame,         omega0 * inertial_frame.z)
lower_leg_left_frame.ang_vel_in(inertial_frame)

In [ ]:
upper_leg_left_frame.set_ang_vel(lower_leg_left_frame,   omega1     * inertial_frame.z)
upper_leg_left_frame.ang_vel_in(inertial_frame)

In [ ]:
hip_frame.set_ang_vel(upper_leg_left_frame,              omega2     * inertial_frame.z)
hip_frame.ang_vel_in(inertial_frame)

In [ ]:
upper_leg_right_frame.set_ang_vel(hip_frame,             omega3     * inertial_frame.z)
upper_leg_right_frame.ang_vel_in(inertial_frame)

In [ ]:
lower_leg_right_frame.set_ang_vel(upper_leg_right_frame, omega4     * inertial_frame.z)
lower_leg_right_frame.ang_vel_in(inertial_frame)

# Linear Velocities

Finally, the linear velocities of the mass centers are needed. Starting at the ankle which has a velocity of 0.

In [ ]:
ankle_left.set_vel(inertial_frame, 0)

Working our way up the chain we can make use of the fact that the joint points are located on two rigid bodies. Any fixed point in a reference frame can be computed if the linear velocity of another point on that frame is known and the frame's angular velocity is known.

$$^I\mathbf{v}^{P_2} = ^I\mathbf{v}^{P_1} + ^I\omega^A \times \mathbf{r}^{\frac{P_2}{P_1}}$$

The `Point.v2pt_theory()` method makes it easy to do this calculation.

In [ ]:
lower_leg_left_mass_center.v2pt_theory(ankle_left, inertial_frame, lower_leg_left_frame)
lower_leg_left_mass_center.vel(inertial_frame)

In [ ]:
knee_left.v2pt_theory(ankle_left, inertial_frame, lower_leg_left_frame)
knee_left.vel(inertial_frame)

In [ ]:
upper_leg_left_mass_center.v2pt_theory(knee_left, inertial_frame, upper_leg_left_frame)
upper_leg_left_mass_center.vel(inertial_frame)

In [ ]:
hip_left.v2pt_theory(knee_left, inertial_frame, upper_leg_left_frame)
hip_left.vel(inertial_frame)

In [ ]:
hip_mass_center.v2pt_theory(hip_left, inertial_frame, hip_frame)
hip_mass_center.vel(inertial_frame)

In [ ]:
hip_right.v2pt_theory(hip_left, inertial_frame, hip_frame)
hip_right.vel(inertial_frame)

In [ ]:
knee_right.v2pt_theory(hip_right, inertial_frame, upper_leg_right_frame)
knee_right.vel(inertial_frame)

In [ ]:
upper_leg_right_mass_center.v2pt_theory(knee_right, inertial_frame, upper_leg_right_frame)
upper_leg_right_mass_center.vel(inertial_frame)

In [ ]:
ankle_right.v2pt_theory(knee_right, inertial_frame, lower_leg_right_frame)
ankle_right.vel(inertial_frame)

In [ ]:
lower_leg_right_mass_center.v2pt_theory(knee_right, inertial_frame, lower_leg_right_frame)
lower_leg_right_mass_center.vel(inertial_frame)

# Masses, Inertia, Rigid Bodies

In [ ]:
lower_leg_mass, upper_leg_mass, hip_mass = symbols('m_L, m_U, m_H')

In [ ]:
lower_leg_inertia, upper_leg_inertia, hip_inertia = symbols('I_Lz, I_Uz, I_Hz')

In [ ]:
lower_leg_left_inertia_dyadic = inertia(lower_leg_left_frame, 0, 0, lower_leg_inertia)
lower_leg_left_central_inertia = (lower_leg_left_inertia_dyadic, lower_leg_left_mass_center)
lower_leg_left_inertia_dyadic.to_matrix(lower_leg_left_frame)

In [ ]:
upper_leg_left_inertia_dyadic = inertia(upper_leg_left_frame, 0, 0, upper_leg_inertia)
upper_leg_left_central_inertia = (upper_leg_left_inertia_dyadic, upper_leg_left_mass_center)
upper_leg_left_inertia_dyadic.to_matrix(upper_leg_left_frame)

In [ ]:
hip_inertia_dyadic = inertia(hip_frame, 0, 0, hip_inertia)
hip_central_inertia = (hip_inertia_dyadic, hip_mass_center)
hip_inertia_dyadic.to_matrix(hip_frame)

In [ ]:
upper_leg_right_inertia_dyadic = inertia(upper_leg_right_frame, 0, 0, upper_leg_inertia)
upper_leg_right_central_inertia = (upper_leg_right_inertia_dyadic, upper_leg_right_mass_center)
upper_leg_right_inertia_dyadic.to_matrix(upper_leg_right_frame)

In [ ]:
lower_leg_right_inertia_dyadic = inertia(lower_leg_right_frame, 0, 0, lower_leg_inertia)
lower_leg_right_central_inertia = (lower_leg_right_inertia_dyadic, lower_leg_right_mass_center)
lower_leg_right_inertia_dyadic.to_matrix(lower_leg_right_frame)

In [ ]:
lower_leg_left = RigidBody('Lower Leg Left', lower_leg_left_mass_center, lower_leg_left_frame,
                      lower_leg_mass, lower_leg_left_central_inertia)

In [ ]:
upper_leg_left = RigidBody('Upper Leg Left', upper_leg_left_mass_center, upper_leg_left_frame,
                      upper_leg_mass, upper_leg_left_central_inertia)

In [ ]:
hip = RigidBody('Hip', hip_mass_center, hip_frame, hip_mass, hip_central_inertia)

In [ ]:
upper_leg_right = RigidBody('Upper Leg Right', upper_leg_right_mass_center, upper_leg_right_frame,
                      upper_leg_mass, upper_leg_right_central_inertia)

In [ ]:
lower_leg_right = RigidBody('Lower Leg Right', lower_leg_right_mass_center, lower_leg_right_frame,
                      lower_leg_mass, lower_leg_right_central_inertia)

# Forces and Torques

In [ ]:
g = symbols('g')

In [ ]:
lower_leg_left_grav_force = (lower_leg_left_mass_center, -lower_leg_mass * g * inertial_frame.y)
upper_leg_left_grav_force = (upper_leg_left_mass_center, -upper_leg_mass * g * inertial_frame.y)
hip_grav_force = (hip_mass_center, -hip_mass * g * inertial_frame.y)
upper_leg_right_grav_force = (upper_leg_right_mass_center, -upper_leg_mass * g * inertial_frame.y)
lower_leg_right_grav_force = (lower_leg_right_mass_center, -lower_leg_mass * g * inertial_frame.y)

In [ ]:
ankle_torque0, knee_torque0, hip_torque0, hip_torque1, knee_torque1, ankle_torque1 = dynamicsymbols('T_a0, T_k0, T_h0, T_h1, T_k1, T_a1')

In [ ]:
lower_leg_left_torque_vector = ankle_torque0 * inertial_frame.z - knee_torque0 * inertial_frame.z
upper_leg_left_torque_vector = knee_torque0 * inertial_frame.z - hip_torque0 * inertial_frame.z
hip_left_torque_vector = hip_torque0 * inertial_frame.z - hip_torque1 * inertial_frame.z
hip_right_torque_vector = hip_torque1 * inertial_frame.z - knee_torque1 * inertial_frame.z
upper_leg_right_torque_vector = knee_torque1 * inertial_frame.z - ankle_torque1 * inertial_frame.z
lower_leg_right_torque_vector = ankle_torque1 * inertial_frame.z

In [ ]:
lower_leg_left_torque = (lower_leg_left_frame, lower_leg_left_torque_vector)
upper_leg_left_torque = (upper_leg_left_frame, upper_leg_left_torque_vector)
hip_left_torque = (hip_frame, hip_left_torque_vector)
hip_right_torque = (hip_frame, hip_right_torque_vector)
upper_leg_right_torque = (upper_leg_right_frame, upper_leg_right_torque_vector)
lower_leg_right_torque = (lower_leg_right_frame, lower_leg_right_torque_vector)

# Equations of Motion

In [ ]:
coordinates = [theta0, theta1, theta2, theta3, theta4]
coordinates

In [ ]:
speeds = [omega0, omega1, omega2, omega3, omega4]
speeds

In [ ]:
kinematical_differential_equations

In [ ]:
kane = KanesMethod(inertial_frame, coordinates, speeds, kinematical_differential_equations)

In [ ]:
loads = [lower_leg_left_grav_force,
         upper_leg_left_grav_force,
         hip_grav_force, 
         upper_leg_right_grav_force,
         lower_leg_right_grav_force,
         lower_leg_left_torque,
         upper_leg_left_torque,
         hip_left_torque,
         hip_right_torque,
         upper_leg_right_torque,
         lower_leg_right_torque]
loads

In [ ]:
bodies = [lower_leg_left, upper_leg_left, hip, upper_leg_right, lower_leg_right]
bodies

In [ ]:
fr, frstar = kane.kanes_equations(loads, bodies)

In [ ]:
#trigsimp(fr + frstar)

Keep in mind that out utlimate goal is to have the equations of motion in first order form:

$$ \dot{\mathbf{x}} = \mathbf{g}(\mathbf{x}, t) $$

The equations of motion are linear in terms of the derivatives of the generalized speeds and the `KanesMethod` class automatically puts the equations in a more useful form for the next step of numerical simulation:

$$ \mathbf{M}(\mathbf{x}, t)\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x}, t) $$

Note that

$$ \mathbf{g} = \mathbf{M}^{-1}(\mathbf{x}, t) \mathbf{f}(\mathbf{x}, t) $$

and that $\mathbf{g}$ can be computed analytically but for non-toy problems, it is best to do this numerically. So we will simply generate the $\mathbf{M}$ and $\mathbf{f}$ matrices for later use.

The mass matrix, $\mathbf{M}$, can be accessed with the `mass_matrix` method (use `mass_matrix_full` to include the kinematical differential equations too. We can use `trigsimp` again to make this relatively compact: 

In [ ]:
mass_matrix = trigsimp(kane.mass_matrix_full)
mass_matrix

The right hand side, $\mathbf{f}$, is a vector function of all the non-inertial forces (gyroscopic, external, coriolis, etc):

In [ ]:
forcing_vector = trigsimp(kane.forcing_full)
forcing_vector

# Simulation

In [ ]:
constants = [lower_leg_length,
             upper_leg_length,
             hip_length,
             
             lower_leg_left_com_length,
             upper_leg_left_com_length,
             hip_com_length,
             upper_leg_right_com_length,
             lower_leg_right_com_length,
             
             lower_leg_mass,
             upper_leg_mass,
             hip_mass,
             
             lower_leg_inertia,
             upper_leg_inertia,
             hip_inertia,
             
             g]
constants

In [ ]:
numerical_constants = array([0.42,  # lower_leg_length [m]
                             0.54, # upper_leg_length [m]
                             0.2, # hip_length
                             
                             0.21,  # lower_leg_com_length [m]
                             0.27,  # upper_leg_com_length
                             0.10,  # hip_com_length
                             0.27,  # upper_leg_com_length
                             0.21,  # lower_leg_com_length [m]
                             
                             1.0,  # lower_leg_mass [kg]
                             1.5,  # upper_leg_mass [kg]
                             2.0,  # hip_mass [kg]
                             
                             0.1,  # lower_leg_inertia [kg*m^2]                             
                             0.2,  # upper_leg_inertia [kg*m^2]
                             0.1,  # hip_inertia [kg*m^2]
                             
                             9.81],  # acceleration due to gravity [m/s^2]
                            ) 
numerical_constants

In [ ]:
coordinates = [theta0, theta1, theta2, theta3, theta4]
coordinates

In [ ]:
speeds = [omega0, omega1, omega2, omega3, omega4]
speeds

In [ ]:
specified = [ankle_torque0, knee_torque0, hip_torque0, hip_torque1, knee_torque1, ankle_torque1]

In [ ]:
right_hand_side = generate_ode_function(forcing_vector, coordinates,
                                        speeds, constants,
                                        mass_matrix=mass_matrix,
                                        specifieds=specified)

In [ ]:
x0 = zeros(10)
x0

In [ ]:
x0[0] = deg2rad(60)
x0[1] = deg2rad(-30)
x0[2] = deg2rad(-30)
x0[3] = deg2rad(-30)
x0[4] = deg2rad(-30)
x0

In [ ]:
numerical_specified = zeros(6)

args = {'constants': numerical_constants,
        'specified': numerical_specified}
frames_per_sec = 60
final_time = 5.0

t = linspace(0.0, final_time, final_time * frames_per_sec)

In [ ]:
right_hand_side(x0, 0.0, numerical_specified, numerical_constants)

In [ ]:
y = odeint(right_hand_side, x0, t, args=(numerical_specified, numerical_constants))
y.shape

In [ ]:
plot(t, rad2deg(y[:, :5]))
xlabel('Time [s]')
ylabel('Angle [deg]')
legend(["${}$".format(vlatex(c)) for c in coordinates])

In [ ]:
plot(t, rad2deg(y[:, 5:]))
xlabel('Time [s]')
ylabel('Angular Rate [deg/s]')
legend(["${}$".format(vlatex(s)) for s in speeds])

# Visualization

In [ ]:
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point
visualization_pub = rospy.Publisher('/visualization_marker', Marker, queue_size=1)
msg = Marker()
msg.header.frame_id = "world"
msg.header.stamp = rospy.Time.now()
msg.type = Marker.ARROW
msg.scale.x = 0.005
msg.scale.y = 0.05
msg.scale.z = 0.05
msg.color.a = 0.5
msg.color.r = 1.0
msg.color.g = 0.0
msg.color.b = 0.0

In [ ]:
for i in range(0,int(final_time * frames_per_sec)):#
    values = {lower_leg_length: 0.4, upper_leg_length: 0.54, hip_length: 0.2, theta0: y[i, 0], theta1: y[i, 1], theta2: y[i, 2], theta3: y[i, 3], theta4: y[i, 4]}
    
    ankle_left_pos = ankle_left_pos_equation.subs(values).to_matrix(inertial_frame)
    knee_left_pos = knee_left_pos_equation.subs(values).to_matrix(inertial_frame)
    hip_left_pos = hip_left_pos_equation.subs(values).to_matrix(inertial_frame)
    hip_right_pos = hip_right_pos_equation.subs(values).to_matrix(inertial_frame)
    knee_right_pos = knee_right_pos_equation.subs(values).to_matrix(inertial_frame)
    ankle_right_pos = ankle_right_pos_equation.subs(values).to_matrix(inertial_frame)
    
    msg.id = 0
    del msg.points[:]
    msg.points.append(Point(ankle_left_pos[0],ankle_left_pos[1],ankle_left_pos[2]))
    msg.points.append(Point(knee_left_pos[0],knee_left_pos[1],knee_left_pos[2]))
    visualization_pub.publish(msg)
    
    msg.id = 1
    del msg.points[:]
    msg.points.append(Point(knee_left_pos[0],knee_left_pos[1],knee_left_pos[2]))
    msg.points.append(Point(hip_left_pos[0],hip_left_pos[1],hip_left_pos[2]))
    visualization_pub.publish(msg)
    
    msg.id = 2
    del msg.points[:]
    msg.points.append(Point(hip_left_pos[0],hip_left_pos[1],hip_left_pos[2]))
    msg.points.append(Point(hip_right_pos[0],hip_right_pos[1],hip_right_pos[2]))
    visualization_pub.publish(msg)
    
    msg.id = 3
    del msg.points[:]
    msg.points.append(Point(hip_right_pos[0],hip_right_pos[1],hip_right_pos[2]))
    msg.points.append(Point(knee_right_pos[0],knee_right_pos[1],knee_right_pos[2]))
    visualization_pub.publish(msg)
    
    msg.id = 4
    del msg.points[:]
    msg.points.append(Point(knee_right_pos[0],knee_right_pos[1],knee_right_pos[2]))
    msg.points.append(Point(ankle_right_pos[0],ankle_right_pos[1],ankle_right_pos[2]))
    visualization_pub.publish(msg)
    
